In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import sys, os
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
import time
from LandMPC.MPCLandControl import MPCLandControl
from src.rocket import Rocket, perturb_rocket
from src.pos_rocket_vis import *
from LandMPC.utils import NB_X, X, Y, Z, GAMA

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir,"rocket.yaml")

# Rocket setup
Ts  = 1/20
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
rocket.mass = 1.7 # Do not change!!!

# Visualization setup
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1

In [ ]:
sim_time = 20 # simulation length in seconds
x0 = np.array(NB_X*[0])
x0[X] = 3
x0[Y] = 2
x0[Z] = 10
x0[GAMA] = np.deg2rad(30)
x_ref = np.array(NB_X*[0])
x_ref[X] = 1
x_ref[Y] = 0
x_ref[Z] = 3
x_ref[GAMA] = np.deg2rad(0)
xs, us = rocket.trim(x_ref)
print("Linearization around a steady state:")
print("x_ref = ", x_ref)
print("xs = ", xs)
print("us = ", us)
A, B = rocket.linearize(xs, us)

# MPC parameters
H = 5
# Merge four linear mpc
mpc = MPCLandControl().new_controller(rocket, Ts, H, x_ref=x_ref)

In [ ]:
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol = rocket.simulate_land(mpc, sim_time, H, x0)
vis.animate(t_cl[:-1], x_cl[:,:-1], u_cl, T_ol=t_ol[...,:-1], X_ol=x_ol, U_ol=u_ol);


In [ ]:
plot_static_states_inputs(t_cl[:-1], x_cl[:,:-1], u_cl, xs)
plot_static_states_inputs(t_ol[:-1, 0], x_ol[:,:-1, 0], u_ol[:, :, 0], xs)